In [1]:
import networkx as nx
from types_definition import * 
import vector
from vector import Vector
from rotorconfig import RotorConfig
from rotorgraph import RotorGraph, display_path, all_config_from_recurrent, display_grid
from particleconfig import ParticleConfig
from matrices import Matrix
from arcsum import ArcSum
import numpy as np
from smithnormalform import matrix, snfproblem, z
from otherfunctions import *
import random

import matplotlib.pyplot as plt
from itertools import product

In [15]:
#INITIALISATION, creation du graphe complet 
n=4
sinks = {0,n-1}

sinks = {0}
list_sinks= list(sinks)

rg= RotorGraph.complete_graph_with_sink(n, sinks)
rotor_graph = rg.remove_sink_out_edges(sinks)


In [16]:
#Générer une config de rotor au hazard

def radom_rotor_config(rotor_graph:RotorGraph)->RotorConfig:

    ##Cyclepush matrice
    MatrixCP = Matrix(rotor_graph.cycle_push_matrix())

    MatrixCP = MatrixCP.to_numpy()

    rho = RotorConfig(rotor_graph)
    for i in range(1,n-1):
        eh= random.randint(0, len(rotor_graph.rotor_order[i])-1)
        rho[i]= rotor_graph.rotor_order[i][eh]

    #Transformer la config de rotor en somme d'arc
    arc_sum = ArcSum(rho)


    #destalibiser la config de rotor avec des éléments la MCP
    vect_pertube = 0*MatrixCP[0]
    for cp in MatrixCP:
        coef= random.randint(-2, 2)
        vect_pertube += coef*cp 

    i=0
    for edge in rotor_graph.edges:
        arc_sum = arc_sum + ArcSum({edge: vect_pertube[i]})
        i+=1
    
    return arc_sum


#Test
arc_sum = radom_rotor_config(rotor_graph)
print(arc_sum)


{(3, 2, 0): 3, (1, 2, 0): 6, (3, 0, 0): 4, (2, 1, 0): 1, (3, 1, 0): -6, (1, 0, 0): -5, (2, 0, 0): -2, (1, 3, 0): 0, (2, 3, 0): 2}


In [17]:
#Ajouter l'action d'envoyer une particule dans un puits à une somme d'arc.

def send_to_sink(n: int,rotor_graph:RotorGraph, arc_sum:ArcSum,node= Node, sink=Node)-> ArcSum:
    """
    In the complete graph
    Gives the sums of arcs obtained by routing a particle on the given Node to the given Sink
    """
    if (1<= node <= n-2) and (sink==0 or sink == n-1):
        edge= (node,sink,0)
    else:
        raise ValueError("node or sink not valid")
    result = arc_sum + ArcSum({rotor_graph.turn(edge):1}) - ArcSum({edge:1})
    
    return result

In [18]:

#recuper les arcs positifs dans la somme
def positif_arc(arc_sum:ArcSum)->ArcSum:
    positif_el= ArcSum()
    for edge, count in arc_sum.configuration.items():
        if count > 0:
            positif_el = positif_el + ArcSum({edge:1})
    return positif_el

#prendre les predecesseur des acrs positifs
def pred_arc(rotor_graph:RotorGraph, arc_sum:ArcSum)->ArcSum:
    pred_el= ArcSum()
    for edge, count in arc_sum.configuration.items():
        pred = rotor_graph.reverse_turn(edge)
        pred_el = pred_el + ArcSum({(pred[1], pred[0], 0):1})
        
    return pred_el






In [28]:
#circulation formée par les arcs positifs et leur predecesseur

def circulation_in_sum(rotor_graph:RotorGraph, arc_sum:ArcSum)-> list[tuple]:
    pos=positif_arc( arc_sum)
    pred= pred_arc(rotor_graph,pos)
    pos_and_pred= pos+pred
    circulations = find_circulation2(pos_and_pred)

    if not circulations:
        return None

    circulations_pos_pred = []

    for circulation in circulations:
        posc = ArcSum()
        predc = ArcSum()
        for edge in circulation.configuration.keys():
            if edge in pos.configuration.keys():
                posc = posc + ArcSum({edge:1})
            else:
                predc = predc + ArcSum({(edge[1], edge[0], 0):-1})
        circulations_pos_pred.append((posc,predc))


    #prufication des circulations

    #Supprimer les circulations portant sur le meme arc en positif et negatif
    for circulation in circulations_pos_pred:
        pos_c = circulation[0]
        pred_c = circulation[1]
        sum_c =pos_c + pred_c
        if len(list(sum_c.configuration.keys())) == 1 and list(sum_c.configuration.values())==[0]:
            circulations_pos_pred.remove(circulation)


    #Supprimer les circulations portant sur un arc en positif ayant un prédecesseur en negatif 
    #sans que sa valeur soit supérieur à 2
    for circulation in circulations_pos_pred:
        pos_c = circulation[0]
        pred_c = circulation[1]

        for edge in pos_c.configuration.keys():
            pred_edge = rotor_graph.reverse_turn(edge)
            if pred_edge in pred_c.configuration.keys() and dict(arc_sum.configuration.items())[edge]<2:
                circulations_pos_pred.remove(circulation)

    return circulations_pos_pred






#push d'une circulation
def push_circulation_in_sum(rotor_graph:RotorGraph, arc_sum:ArcSum, circulations_pos_pred:tuple):
    
    posc = circulations_pos_pred[0]
    predc = circulations_pos_pred[1]
    
    if posc:

        for edge, count in posc.configuration.items():
            arc_sum += ArcSum({rotor_graph.turn(edge):count}) - ArcSum({edge:count})
            
    if predc:
        for edge, count in predc.configuration.items():
            arc_sum += ArcSum({rotor_graph.turn(edge):count}) - ArcSum({edge:count})
        

    #arc_sum = ArcSum(arc_sum)

    return arc_sum
    
# edges_on_node += ArcSum({rotor_graph.turn(edge_2):count_2}) - ArcSum({edge_2:count_2})

# #test
a=circulation_in_sum(rotor_graph, arc_sum)


print(arc_sum)
print(a[0])
# print(arc_sum)
# #print(a[0])
print((push_circulation_in_sum(rotor_graph, arc_sum, a[0])))




{(3, 2, 0): 0, (1, 2, 0): 1, (3, 0, 0): -1, (2, 1, 0): -5, (3, 1, 0): 2, (1, 0, 0): -1, (2, 0, 0): 6, (1, 3, 0): 1, (2, 3, 0): 0}
({(2, 0, 0): 1}, {(3, 0, 0): -1, (2, 3, 0): -1})
{(1, 2, 0): 1, (3, 0, 0): 0, (2, 1, 0): -4, (2, 3, 0): 1, (3, 1, 0): 1, (1, 0, 0): -1, (2, 0, 0): 4, (1, 3, 0): 1, (3, 2, 0): 0}


In [29]:
##renvoi  un dico avec la liste des arcs sur chaque sommet
def edges_per_nodes(rotor_graph: RotorGraph, arc_sum: ArcSum ):

    edges_per_nodes = dict()
    for edge,count in arc_sum.configuration.items():
        if edge[0] in rotor_graph.nodes :
            if edge[0] not in edges_per_nodes.keys():
                edges_per_nodes[edge[0]]= ArcSum()
            edges_per_nodes[edge[0]] += ArcSum({edge:count})

    return edges_per_nodes


#Test

#print(arc_sum)
#print(edges_per_nodes(rotor_graph, arc_sum ))

In [30]:

node=1


negative_edge= (1,3,0)
order_list = rotor_graph.rotor_order[node]
indice = order_list.index(negative_edge)
print(order_list)
print(indice)

[(1, 0, 0), (1, 2, 0), (1, 3, 0)]
2


In [31]:
##renvoyer une circulation contenant un arc donné en positive
def arc_in_circulation(circulation_in_sum:list[tuple], arc:Edge)-> tuple:
    

    if not circulation_in_sum:
        return None

    arc_in_circulation = []
    for circulation in circulation_in_sum:
        pos = circulation[0]
        pred = circulation [1]
        if arc in pos.configuration.keys():
            arc_in_circulation = circulation
            break
        elif arc in pred.configuration.keys():
            arc_in_circulation = circulation
            break 
            
    
    if arc_in_circulation == []:
        return None
    
    return arc_in_circulation


a=circulation_in_sum(rotor_graph, arc_sum)
print(a)
arc= (1,0,0)

#print(arc_sum)
print(arc_in_circulation(a, arc))



[({(2, 0, 0): 1}, {(3, 0, 0): -1, (2, 3, 0): -1}), ({(1, 3, 0): 1, (2, 0, 0): 1}, {(1, 0, 0): -1, (2, 3, 0): -1}), ({(1, 3, 0): 1, (3, 1, 0): 1}, {})]
({(1, 3, 0): 1, (2, 0, 0): 1}, {(1, 0, 0): -1, (2, 3, 0): -1})


In [32]:
def circulation_to_push(rotor_graph:RotorGraph, arc_sum:ArcSum)->ArcSum:
    
    #edges_on_node= edges_per_nodes(rotor_graph, arc_sum )
    #trouver un arc negatif
    negative_edge = arc_sum.first_negative_edge()

    

    if negative_edge:
        node = negative_edge[0]
        #print(negative_edge, node)
        #considerer les arcs sur la queue de l'arc négatif
        #arc_on_node = edges_on_node[node]
        #print("arc on node", arc_on_node)

        #trouver l'arc positif le plus proche precedant l'arc negatif dans l'ordre de rotor
        order_list = rotor_graph.rotor_order[node]
        #print(order_list)
        indice = order_list.index(negative_edge)
        
        
        for i in range(0, len(order_list)):
            #print(i)
            indice_proche_p = indice - i
            edgep = order_list[indice_proche_p]

            indice_proche_n = (indice + i)%len(order_list)
            edgen = order_list[indice_proche_n]
            
        
            if dict(arc_sum.configuration.items())[edgep] > 0 :
                arc_proche = edgep

                
                circulation = arc_in_circulation(circulation_in_sum(rotor_graph, arc_sum), arc_proche)
                #print(arc_proche,circulation)
                if circulation :
                    break

            if dict(arc_sum.configuration.items())[edgen] > 0 :
                arc_proche = rotor_graph.reverse_turn(edgen) 
                circulation = arc_in_circulation(circulation_in_sum(rotor_graph, arc_sum), arc_proche)
                #print(arc_proche,circulation)
                if circulation :
                    break

                
                

        
        #circulation dans la somme
        return circulation
    else:
        return None 


#Test

# sum1= ArcSum({(1, 0, 0): 0, (1, 2, 0): 0, (2, 1, 0): -3, (1, 3, 0): 1, (2, 0, 0): 0, (2, 3, 0): 4})
           
# a=circulation_to_push(rotor_graph, sum1)

# print(a)

# print(circulation_in_sum(rotor_graph, sum1))

# if a:
#     print(a[0])
# else:
#     print("le truc est vide")


In [39]:
 #Tentative de fonction de stabilisation:
def stab_arc(rotor_graph:RotorGraph, arc_sum:ArcSum)->ArcSum:
    #print(arc_sum)
    while circulation_to_push(rotor_graph, arc_sum):
        
        circulation = circulation_to_push(rotor_graph, arc_sum)
        print(circulation)
        arc_sum = push_circulation_in_sum(rotor_graph, arc_sum, circulation)

        
        print(arc_sum)
    
    return arc_sum


#arc_sum = radom_rotor_config(rotor_graph)

print(arc_sum)
print(stab_arc(rotor_graph, arc_sum))
Prob=ArcSum({(1, 0, 0): 0, (1, 2, 0): 3, (2, 1, 0): -1, (1, 3, 0): -2, (2, 0, 0): 0, (2, 3, 0): 2})

{(3, 2, 0): 2, (1, 2, 0): 8, (3, 0, 0): 1, (2, 1, 0): -7, (3, 1, 0): -2, (1, 0, 0): -6, (2, 0, 0): 8, (1, 3, 0): -1, (2, 3, 0): 0}
({(3, 2, 0): 1, (2, 0, 0): 1}, {(1, 0, 0): -1, (3, 1, 0): -1})
{(3, 2, 0): 0, (1, 2, 0): 7, (3, 0, 0): 2, (2, 1, 0): -6, (3, 1, 0): -1, (1, 0, 0): -5, (2, 0, 0): 7, (1, 3, 0): -1, (2, 3, 0): 0}
({(2, 0, 0): 1, (1, 2, 0): 1}, {(1, 0, 0): -1})
{(1, 2, 0): 5, (3, 0, 0): 2, (2, 1, 0): -5, (2, 3, 0): 0, (3, 1, 0): -1, (1, 0, 0): -4, (2, 0, 0): 6, (1, 3, 0): 0, (3, 2, 0): 0}
({(2, 0, 0): 1, (1, 2, 0): 1}, {(1, 0, 0): -1})
{(3, 2, 0): 0, (1, 2, 0): 3, (3, 0, 0): 2, (2, 1, 0): -4, (3, 1, 0): -1, (1, 0, 0): -3, (2, 0, 0): 5, (1, 3, 0): 1, (2, 3, 0): 0}
({}, {(3, 2, 0): -1, (2, 3, 0): -1})
{(3, 2, 0): 1, (1, 2, 0): 3, (3, 0, 0): 1, (2, 1, 0): -4, (3, 1, 0): -1, (1, 0, 0): -3, (2, 0, 0): 4, (1, 3, 0): 1, (2, 3, 0): 1}
({(1, 3, 0): 1, (3, 2, 0): 1, (2, 0, 0): 1}, {(1, 0, 0): -1})
{(3, 2, 0): 0, (1, 2, 0): 2, (3, 0, 0): 2, (2, 1, 0): -3, (3, 1, 0): -1, (1, 0, 0): -1, (2

KeyboardInterrupt: 

In [40]:
#test pour debugger

arc_sum= ArcSum({(3, 2, 0): 0, (1, 2, 0): 3, (3, 0, 0): 2, (2, 1, 0): -4, 
                 (3, 1, 0): -1, (1, 0, 0): -3, (2, 0, 0): 5, (1, 3, 0): 1, (2, 3, 0): 0}
)

print(arc_sum)

print(circulation_to_push(rotor_graph, arc_sum))

print(circulation_in_sum(rotor_graph, arc_sum))

pos=positif_arc( arc_sum)
pred= pred_arc(rotor_graph,pos)
pos_and_pred= pos+pred

print(pos_and_pred)
circulations = find_circulation2(pos_and_pred)

print(circulations)


{(3, 2, 0): 0, (1, 2, 0): 3, (3, 0, 0): 2, (2, 1, 0): -4, (3, 1, 0): -1, (1, 0, 0): -3, (2, 0, 0): 5, (1, 3, 0): 1, (2, 3, 0): 0}
({}, {(3, 2, 0): -1, (2, 3, 0): -1})
[({(3, 0, 0): 1, (1, 3, 0): 1}, {(1, 0, 0): -1}), ({(3, 0, 0): 1, (1, 2, 0): 1}, {(1, 0, 0): -1, (3, 2, 0): -1}), ({}, {(3, 2, 0): -1, (2, 3, 0): -1})]
{(3, 2, 0): 1, (0, 1, 0): 1, (1, 2, 0): 1, (3, 0, 0): 1, (2, 1, 0): 1, (2, 0, 0): 1, (1, 3, 0): 1, (2, 3, 0): 1}
[{(0, 1, 0): 1, (3, 0, 0): 1, (1, 3, 0): 1}, {(0, 1, 0): 1, (1, 2, 0): 1, (3, 0, 0): 1, (2, 3, 0): 1}, {(3, 2, 0): 1, (2, 3, 0): 1}, {(3, 2, 0): 1, (2, 1, 0): 1, (1, 3, 0): 1}]
